In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator

spark = SparkSession.builder.master("local[*]").getOrCreate()
filename_data = 'datasetelectro.csv'
csv = spark.read.csv(filename_data, inferSchema=True, header=True)
csv = csv.withColumn('mileage', csv.mileage.cast(IntegerType()))
csv.show(10)

+---+----------+---------+----------+----+-------+---------+------------+--------------+------+-------+
|_c0|     brand|    model|  offer_id|year|mileage|fuel_type|engine_power|  transmission| wheel|  price|
+---+----------+---------+----------+----+-------+---------+------------+--------------+------+-------+
|  0|    Toyota|     Vitz|1102756899|2001|   6236|   Бензин|        70.0|автоматическая|Правый| 230000|
|  1|LADA (ВАЗ)|     2107|1102754237|1995|  96000|   Бензин|        75.0|  механическая| Левый|  35000|
|  2|    Daewoo|   Nubira|1102763224|1998|  50000|   Бензин|       106.0|  механическая| Левый|  45000|
|  3| Chevrolet|     Aveo|1102757199|2014| 119120|   Бензин|       115.0|  механическая| Левый| 475000|
|  4| Chevrolet|  Lacetti|1102760232|2011|  87650|   Бензин|       109.0|  механическая| Левый| 479000|
|  5|       Kia|   Cerato|1102753937|2010| 160000|   Бензин|       126.0|  механическая| Левый| 520000|
|  6|    Nissan| Wingroad|1102757232|2011| 132300|   Бензин|    

In [2]:
splits = csv.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("price", "truePrice")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 181586  Testing Rows: 77813


In [3]:
strIdx = StringIndexer(inputCols = ['brand', 'model', 'fuel_type','transmission', 'wheel'], 
                       outputCols = ['brand_index', 'model_index', 'fuel_type_index', 'transmission_index', 'wheel_index'], 
                       handleInvalid = "keep")
catVect = VectorAssembler(inputCols = ['brand_index', 'model_index', 'fuel_type_index','transmission_index', 'wheel_index'],
                          outputCol="features_cat")
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), 
                       outputCol = "features_index", 
                       handleInvalid = "keep")
numVect = VectorAssembler(inputCols = ["mileage",'engine_power', 'year'], 
                          outputCol="features_num", 
                          handleInvalid = "keep")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), 
                      outputCol="features_norm")
featVect = VectorAssembler(inputCols=["features_index", "features_norm"], 
                           outputCol="features", 
                           handleInvalid = "keep")
rfr = RandomForestRegressor(featuresCol = 'features', 
                      labelCol='price',
                      numTrees = 10,
                      maxDepth=2,
                      maxBins = 181834)
pipeline = Pipeline(stages=[strIdx, catVect, catIdx, numVect, minMax, featVect, rfr])

In [4]:
pipelineModel = pipeline.fit(train)

In [5]:
pred_df = pipelineModel.transform(test)
pred_df.select("features", "prediction", "truePrice").show()

+--------------------+------------------+---------+
|            features|        prediction|truePrice|
+--------------------+------------------+---------+
|[3.0,150.0,0.0,1....| 352327.1654304767|   230000|
|(8,[1,5,6,7],[9.0...|181187.53397100177|    35000|
|[18.0,465.0,0.0,0...|239247.00039099803|    45000|
|[4.0,54.0,0.0,0.0...| 596291.4979474199|   520000|
|[4.0,29.0,0.0,0.0...| 717731.4296246916|   875000|
|[12.0,42.0,0.0,1....| 661451.7150089791|   595000|
|[6.0,21.0,0.0,0.0...|207808.07458512558|   158000|
|[3.0,99.0,0.0,1.0...| 592913.6587095854|   530000|
|[5.0,13.0,0.0,1.0...|  717396.273998782|   877700|
|[9.0,138.0,0.0,1....| 733412.3300384547|   750000|
|[3.0,148.0,3.0,1....| 722951.2637245711|  1100000|
|[9.0,8.0,0.0,0.0,...| 433479.8933566821|   470000|
|[17.0,56.0,0.0,2....| 755847.5233929649|   985000|
|[7.0,1.0,0.0,3.0,...| 546648.1337267073|   520000|
|[13.0,165.0,1.0,1...|514533.90636527224|   500000|
|[1.0,164.0,1.0,0....|321601.93818566843|   320000|
|[6.0,21.0,0

In [6]:
from pyspark.ml.evaluation import RegressionEvaluator

regressionEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="truePrice", metricName="rmse")

In [7]:
# RMSE
rmse = regressionEvaluator.evaluate(pred_df)
print(f"The RMSE for the random forest regression model is {rmse:0.2f}")
# MSE
mse = regressionEvaluator.setMetricName("mse").evaluate(pred_df)
print(f"The MSE for the random forest regression model is {mse:0.2f}")
# R2
r2 = regressionEvaluator.setMetricName("r2").evaluate(pred_df)
print(f"The R2 for the random forest regression model is {r2:0.2f}")
# MAE
mae = regressionEvaluator.setMetricName("mae").evaluate(pred_df)
print(f"The MAE for the random forest regression model is {mae:0.2f}")

The RMSE for the random forest regression model is 181040.83
The MSE for the random forest regression model is 32775781158.95
The R2 for the random forest regression model is 0.69
The MAE for the random forest regression model is 131134.55


In [8]:
param_grid = ParamGridBuilder().\
    addGrid(rfr.numTrees, [10, 15, 20]).\
    addGrid(rfr.maxDepth, [1, 2, 4]).\
    addGrid(rfr.maxBins , [181834, 362432, 724864]).\
    build()

In [9]:
cv = CrossValidator(estimator=pipeline, \
                    estimatorParamMaps=param_grid, \
                    evaluator=RegressionEvaluator(
                                predictionCol="prediction", \
                                labelCol="price", \
                                metricName="rmse"), \
                    numFolds=2)

In [10]:
cv_model = cv.fit(train)

In [11]:
newPrediction = cv_model.transform(test)

In [12]:
# RMSE
rmse = regressionEvaluator.evaluate(newPrediction)
print(f"The RMSE for the random forest regression model is {rmse:0.2f}")
# MSE
mse = regressionEvaluator.setMetricName("mse").evaluate(newPrediction)
print(f"The MSE for the random forest regression model is {mse:0.2f}")
# R2
r2 = regressionEvaluator.setMetricName("r2").evaluate(newPrediction)
print(f"The R2 for the random forest regression model is {r2:0.2f}")
# MAE
mae = regressionEvaluator.setMetricName("mae").evaluate(newPrediction)
print(f"The MAE for the random forest regression model is {mae:0.2f}")

The RMSE for the random forest regression model is 92305.63
The MSE for the random forest regression model is 17832910506.74
The R2 for the random forest regression model is 0.83
The MAE for the random forest regression model is 92305.63


2 часть

In [13]:
csv = csv.drop(csv._c0).withColumn('label', when(col('mileage') >= 150942.9, 1).otherwise(0))
csv.drop(csv.mileage).show()

+----------+---------+----------+----+---------+------------+----------------+------+-------+-----+
|     brand|    model|  offer_id|year|fuel_type|engine_power|    transmission| wheel|  price|label|
+----------+---------+----------+----+---------+------------+----------------+------+-------+-----+
|    Toyota|     Vitz|1102756899|2001|   Бензин|        70.0|  автоматическая|Правый| 230000|    0|
|LADA (ВАЗ)|     2107|1102754237|1995|   Бензин|        75.0|    механическая| Левый|  35000|    0|
|    Daewoo|   Nubira|1102763224|1998|   Бензин|       106.0|    механическая| Левый|  45000|    0|
| Chevrolet|     Aveo|1102757199|2014|   Бензин|       115.0|    механическая| Левый| 475000|    0|
| Chevrolet|  Lacetti|1102760232|2011|   Бензин|       109.0|    механическая| Левый| 479000|    0|
|       Kia|   Cerato|1102753937|2010|   Бензин|       126.0|    механическая| Левый| 520000|    1|
|    Nissan| Wingroad|1102757232|2011|   Бензин|       109.0|        вариатор|Правый| 584000|    0|


In [14]:
splits = csv.randomSplit([0.85, 0.15])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
print("Training Rows:", train.count(), " Testing Rows:", test.count())

Training Rows: 220621  Testing Rows: 38778


In [15]:
strIdx = StringIndexer(inputCols = ['brand', 'model', 'fuel_type','transmission'], 
                       outputCols = ['brand_index', 'model_index', 'fuel_type_index', 'transmission_index',], 
                       handleInvalid = "keep")
catVect = VectorAssembler(inputCols = ['brand_index', 'model_index', 'fuel_type_index', 'transmission_index',], 
                          outputCol="features_cat")
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), 
                       outputCol = "features_index", 
                       handleInvalid = "keep")
numVect = VectorAssembler(inputCols = ["mileage",'engine_power', 'year'], 
                          outputCol="features_num", 
                          handleInvalid = "keep")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), 
                      outputCol="features_norm")
featVect = VectorAssembler(inputCols=["features_index", "features_norm"], 
                           outputCol="features", 
                           handleInvalid = "keep")
lr = LogisticRegression(labelCol="label", 
                        featuresCol="features", 
                        maxIter=10,
                        regParam=0.3)
pipeline = Pipeline(stages=[strIdx, catVect, catIdx, numVect, minMax, featVect, lr])

In [16]:
pipelineModel = pipeline.fit(train)

In [17]:
pred_df = pipelineModel.transform(test)
pred_df.select("features", "prediction", "trueLabel").show()

+--------------------+----------+---------+
|            features|prediction|trueLabel|
+--------------------+----------+---------+
|[56.0,531.0,0.0,0...|       1.0|        1|
|[56.0,531.0,1.0,0...|       1.0|        1|
|[56.0,1386.0,0.0,...|       0.0|        0|
|[16.0,127.0,0.0,0...|       1.0|        1|
|[16.0,127.0,1.0,0...|       0.0|        0|
|[16.0,127.0,1.0,0...|       1.0|        1|
|[16.0,127.0,0.0,0...|       1.0|        1|
|[16.0,127.0,0.0,0...|       1.0|        1|
|[16.0,127.0,0.0,0...|       1.0|        1|
|[16.0,127.0,0.0,0...|       1.0|        1|
|[16.0,577.0,0.0,0...|       1.0|        1|
|[16.0,118.0,0.0,0...|       1.0|        1|
|[16.0,118.0,0.0,0...|       1.0|        1|
|[16.0,118.0,0.0,0...|       1.0|        1|
|[16.0,118.0,0.0,0...|       1.0|        1|
|[16.0,118.0,0.0,0...|       1.0|        1|
|[16.0,118.0,0.0,0...|       1.0|        1|
|[16.0,118.0,0.0,0...|       1.0|        1|
|[16.0,118.0,0.0,0...|       1.0|        1|
|[16.0,118.0,0.0,0...|       0.0

In [18]:
tp = float(pred_df.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(pred_df.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(pred_df.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(pred_df.filter("prediction == 0.0 AND truelabel == 1").count())
pr = tp / (tp + fp)
re = tp / (tp + fn)
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", pr),
 ("Recall", re),
 ("F1", 2*pr*re/(re+pr))],["metric", "value"])
metrics.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|           14439.0|
|       FP|             334.0|
|       TN|           20680.0|
|       FN|            3325.0|
|Precision| 0.977391186624247|
|   Recall|0.8128236883584778|
|       F1| 0.887543412115438|
+---------+------------------+



In [19]:
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
aur = evaluator.evaluate(pred_df)
print ("AUR = ", aur)

AUR =  0.9894194996518229


In [20]:
paramGrid = ParamGridBuilder().\
    addGrid(lr.maxIter, [10, 20, 30]).\
    addGrid(lr.regParam, [0.3, 0.5, 0.7]).build()
cv = CrossValidator(estimator=pipeline, evaluator=BinaryClassificationEvaluator(metricName='areaUnderPR'), estimatorParamMaps=paramGrid, 
                    numFolds=2)

In [21]:
cv_model = cv.fit(train)

In [22]:
newPrediction = cv_model.transform(test)

In [23]:
# Recalculate confusion matrix
tp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 1").count())
fp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 0").count())
tn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 0").count())
fn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 1").count())
pr2 = tp2 / (tp2 + fp2)
re2 = tp2 / (tp2 + fn2)
metrics2 = spark.createDataFrame([
 ("TP", tp2),
 ("FP", fp2),
 ("TN", tn2),
 ("FN", fn2),
 ("Precision", pr2),
 ("Recall", re2),
 ("F1", 2*pr2*re2/(re2+pr2))],["metric", "value"])
metrics2.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|           14439.0|
|       FP|             334.0|
|       TN|           20680.0|
|       FN|            3325.0|
|Precision| 0.977391186624247|
|   Recall|0.8128236883584778|
|       F1| 0.887543412115438|
+---------+------------------+



In [24]:
# Recalculate the Area Under ROC
evaluator2 = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="prediction", metricName="areaUnderROC")
aur2 = evaluator2.evaluate(newPrediction)
print( "AUR2 = ", aur2)

AUR2 =  0.8984647612821227
